## Initialization

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
import itertools
palette = itertools.cycle(sns.color_palette())
from func import get_data
# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Import Data

In [2]:
# Read the CSV file into a pandas DataFrame and display the first few rows
path_ella = 'C:/Users/ellad/Desktop/MB_ML/Projekt/train_heart.csv'
path_train = './train_heart.csv'
path_test = './test_heart.csv'

col = ['sex','cp','fbs','restecg','exang','slope','ca','thal']

[X_train,y_train,X_test,y_test,age,feature_name] = get_data(False,True)


print("Training Dataset size:", len(X_train))
print("Test Dataset size:", len(X_test))

print('Training data shape: ',X_train.shape)
print('Test data shape: ',X_test.shape)

Training Dataset size: 1025
Test Dataset size: 303
Training data shape:  (1025, 30)
Test data shape:  (303, 30)


In [3]:
X_nn_1 = X_train[:,0:5]
X_nn_2 = X_train[:,7:]
X_nn = np.hstack([X_nn_1,X_nn_2])

X_linear = X_train[:,5:7]

print(X_nn.shape)
print(X_linear.shape)

# print(feature_name)
#  print(feature_name[5])  - Sex 0
#  print(feature_name[6])  - Sex 1
# print(feature_name[5:7])

# print(feature_name[0:5])
# print(feature_name[7:])

# print(np.hstack([feature_name[0:5],feature_name[7:]]).shape)







(1025, 28)
(1025, 2)


## NN Model

In [4]:
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
class FFNN(PyroModule):
    def __init__(self, n_in, n_hidden, n_out):
        super(FFNN, self).__init__()
        
        # Architecture
        self.in_layer = PyroModule[nn.Linear](n_in, n_hidden)
        self.in_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_in]).to_event(2))

        self.h_layer = PyroModule[nn.Linear](n_hidden, n_hidden)
        self.h_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_hidden]).to_event(2))

        self.out_layer = PyroModule[nn.Linear](n_hidden, n_out)
        self.out_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_out, n_hidden]).to_event(2))

        # Activation functions
        self.tanh = nn.Tanh()
        
    def forward(self, X, y=None):
        X = self.tanh(self.in_layer(X))
        X = self.tanh(self.h_layer(X))
        X = self.out_layer(X)
        prediction_mean = X.squeeze(-1)

        """
        with pyro.plate("observations"):
            y = pyro.sample("obs", dist.Categorical(logits=alpha + X.matmul(beta)), obs=y)
        """

        with pyro.plate("observations"):
            y = pyro.sample("obs", dist.Bernoulli(logits=prediction_mean), obs=y)
            
        return y

In [5]:
X = torch.tensor(X_train.astype('float')).float()
y = torch.tensor(y_train).float()


In [6]:
# Define guide function
model = FFNN(n_in=X.shape[1], n_hidden=4, n_out=1)
guide = AutoDiagonalNormal(model)
pyro.clear_param_store()

In [7]:
# Define the number of optimization steps
n_steps = 20000

# Setup the optimizer
adam_params = {"lr": 0.01}
optimizer = Adam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X, y)
    if step % 1000 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 961.0
[1000] ELBO: 383.1
[2000] ELBO: 363.9
[3000] ELBO: 366.8
[4000] ELBO: 353.3
[5000] ELBO: 351.5
[6000] ELBO: 362.2
[7000] ELBO: 371.2
[8000] ELBO: 340.6
[9000] ELBO: 385.7
[10000] ELBO: 346.0
[11000] ELBO: 337.3
[12000] ELBO: 341.5
[13000] ELBO: 368.9
[14000] ELBO: 383.6
[15000] ELBO: 347.0
[16000] ELBO: 342.6
[17000] ELBO: 359.7
[18000] ELBO: 310.3
[19000] ELBO: 352.8


In [8]:
# Prepare test data for Pyro
X_test_tensor = torch.tensor(X_test.astype('float')).float()

from pyro.infer import Predictive
predictive = Predictive(model, guide=guide, num_samples=1000,
                        return_sites=("obs", "_RETURN"))
samples = predictive(X_test_tensor)

samples = samples['obs'].detach().squeeze()
y_hat = np.round(samples.mean(axis=0).numpy())

mae = np.mean(np.abs(y_test - y_hat))
print("MAE:", mae)
print("Accuracy:", 1.0*np.sum((y_hat) == y_test) / len(y_test))


MAE: 0.0231023102310231
Accuracy: 0.976897689768977


## Neural Network + Linear Model

In [9]:
class FFNN_Lin(PyroModule):
    def __init__(self, n_in, n_hidden, n_out):
        super(FFNN_Lin, self).__init__()
        
        # Architecture
        self.in_layer = PyroModule[nn.Linear](n_in, n_hidden)
        self.in_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_in]).to_event(2))

        self.h_layer = PyroModule[nn.Linear](n_hidden, n_hidden)
        self.h_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_hidden]).to_event(2))

        self.out_layer = PyroModule[nn.Linear](n_hidden, n_out)
        self.out_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_out, n_hidden]).to_event(2))

        # Activation functions
        self.tanh = nn.Tanh()
        
    def forward(self, X, y=None):
        X_nn_1 = X[:,0:5]
        X_nn_2 = X[:,7:]
        X_nn = torch.tensor(np.hstack([X_nn_1,X_nn_2]))
        
        #print(X_nn.shape)
        X_nn_old = X[:,1:]
        #print(X_nn_old.shape)

        X_nn = self.tanh(self.in_layer(X_nn))
        X_nn = self.tanh(self.h_layer(X_nn))
        X_nn = self.out_layer(X_nn)
        nn_out = X_nn.squeeze(-1)
        #print('nn_out: ', nn_out.shape)


        beta_lin = pyro.sample("beta", dist.Normal(torch.zeros(2), torch.ones(2)).to_event())
        #print('beta par: ', beta_lin)
        X_linear = X[:,5:7]
        #print('Lin: ', X_linear.shape)

        with pyro.plate("observations"):
            linear_out = X_linear@beta_lin
            #print('Lin out: ', linear_out.shape)
            y = pyro.sample("obs", dist.Normal(nn_out+linear_out, 0.1), obs=y)
            
        return y

In [10]:
# Define guide function
model = FFNN_Lin(n_in=X.shape[1]-2, n_hidden=4, n_out=1) 
guide = AutoDiagonalNormal(model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 20000

# Setup the optimizer
adam_params = {"lr": 0.01}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X, y)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 70933.7
[500] ELBO: 1787.5
[1000] ELBO: 461.8
[1500] ELBO: 278.4
[2000] ELBO: 199.5
[2500] ELBO: 225.0
[3000] ELBO: 94.5
[3500] ELBO: 85.3
[4000] ELBO: 86.8
[4500] ELBO: 139.3
[5000] ELBO: 72.4
[5500] ELBO: 114.1
[6000] ELBO: 139.2
[6500] ELBO: 88.0
[7000] ELBO: 224.3
[7500] ELBO: 259.2
[8000] ELBO: 87.3
[8500] ELBO: 45.1
[9000] ELBO: 164.5
[9500] ELBO: 50.8
[10000] ELBO: 74.5
[10500] ELBO: 52.4
[11000] ELBO: 43.6
[11500] ELBO: 99.8
[12000] ELBO: 129.7
[12500] ELBO: 99.0
[13000] ELBO: 145.5
[13500] ELBO: 92.5
[14000] ELBO: 86.7
[14500] ELBO: 32.5
[15000] ELBO: 91.3
[15500] ELBO: 36.3
[16000] ELBO: -20.5
[16500] ELBO: 64.4
[17000] ELBO: -73.8
[17500] ELBO: 93.5
[18000] ELBO: -54.2
[18500] ELBO: -98.7
[19000] ELBO: -32.9
[19500] ELBO: 181.4


In [11]:
from pyro.infer import Predictive

# Get posterior samples for beta
predictive = Predictive(model, guide=guide, num_samples=1000,
                        return_sites=("beta",))
samples = predictive(X, y)

In [12]:
print("Estimated beta:", samples["beta"].mean(axis=0).detach().numpy()[0])


Estimated beta: [ 0.01049766 -0.02890764]


In [13]:
from pyro.infer import Predictive
predictive = Predictive(model, guide=guide, num_samples=1000,
                        return_sites=("obs", "_RETURN"))
samples = predictive(X_test_tensor)

samples = samples['obs'].detach().squeeze()
y_hat = np.round(samples.mean(axis=0).numpy())

mae = np.mean(np.abs(y_test - y_hat))
print("MAE:", mae)
print("Accuracy:", 1.0*np.sum((y_hat) == y_test) / len(y_test))

MAE: 0.0165016501650165
Accuracy: 0.9834983498349835
